In [22]:
# Стандартизация
import pandas as pd
from sklearn import preprocessing

df = pd.read_csv('data/occupancy_even.csv')

scaler = preprocessing.StandardScaler()

names = df.columns
scaled_data = scaler.fit_transform(df)

df_std = pd.DataFrame(scaled_data, columns=names)


In [23]:
# Создание и обучение классификатора
from sklearn import svm
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

x = df_std.iloc[:, :-1].values  # все колонки до последней - переменные функции
y = df.iloc[:, -1].values  # в качестве результирующего значения берем последнюю колонку

# делим набор данных на обучающий и тестовый
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

classifier = svm.SVC(probability=True)
classifier.fit(x_train, y_train)  # идентифицирует и изучает параметры модели из обучающего набора данных


SVC(probability=True)

In [24]:
# Оценка качества классификатора
y_pred = classifier.predict(x_test)  # прогноз на основе тестовых данных

# классификационная оценка точности
print("Точность:\n", accuracy_score(y_test, y_pred), "\n")

# вычисляет матрицу ошибок для оценки точности классификации
print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred), "\n")


Точность:
 0.9860834990059643 

Матрица ошибок:
 [[244   7]
 [  0 252]] 



In [25]:
# Генератор входных данных
import pandas as pd
import numpy as np

n = 15  # количество входных объектов

# генерируем параметры в диапазоне, большем допустимого интервала
temp = np.random.uniform(low=15.0, high=30.0, size=(n,))
humid = np.random.uniform(low=10.0, high=40.0, size=(n,))
light = np.random.uniform(low=0.0, high=1200.0, size=(n,))
co2 = np.random.uniform(low=300.0, high=2000.0, size=(n,))
humid_ratio = np.random.uniform(low=0.001, high=0.007, size=(n,))
occupancy = np.zeros((n,))

matrix_aux = np.vstack([temp, humid, light, co2, humid_ratio, occupancy])
matrix = np.transpose(matrix_aux)
df_input = pd.DataFrame(matrix, columns=['Temperature', 'Humidity', 'Light', 'CO2', 'HumidityRatio', 'Occupancy'])

print(df_input)


    Temperature   Humidity        Light          CO2  HumidityRatio  Occupancy
0     20.739320  37.362123  1133.283771  1719.478423       0.003316        0.0
1     22.757693  25.605982   827.189941   475.785309       0.002475        0.0
2     23.087478  13.942455   579.711764  1681.450604       0.005413        0.0
3     16.079716  10.781828   742.932432  1858.804613       0.003393        0.0
4     23.077517  15.598680   868.829939  1236.116394       0.006729        0.0
5     16.377933  25.710899   411.790279  1722.082241       0.006454        0.0
6     18.289836  14.175867   495.179394  1072.893834       0.005131        0.0
7     25.202298  34.577246   833.883141   786.305190       0.003823        0.0
8     24.064973  26.107119   372.435697  1400.546016       0.002795        0.0
9     17.450413  13.601289    51.383301  1234.837321       0.005112        0.0
10    19.290350  20.581057   591.311860  1810.408485       0.001690        0.0
11    17.758581  11.368812  1141.822987   845.381973

In [26]:
# Стандартизируем тестовые данные
import pandas as pd

input_scaled = scaler.transform(df_input)

df_input_std = pd.DataFrame(input_scaled, columns=names)
print(df_input_std)


    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
0     -0.809966  4.644972  3.469087  3.099668      -1.288222  -0.963873
1      1.058139 -0.046189  2.268793 -0.977754      -2.613606  -0.963873
2      1.363372 -4.700393  1.298350  2.974995       2.017713  -0.963873
3     -5.122663 -5.961607  1.938392  3.556446      -1.166881  -0.963873
4      1.354153 -4.039494  2.432077  1.514976       4.091565  -0.963873
5     -4.846647 -0.004323  0.639875  3.108205       3.659253  -0.963873
6     -3.077086 -4.607252  0.966871  0.979854       1.573359  -0.963873
7      3.320743  3.533697  2.295039  0.040279      -0.488040  -0.963873
8      2.268092  0.153784  0.485553  2.054055      -2.108631  -0.963873
9     -3.854014 -4.836532 -0.773398  1.510782       1.543013  -0.963873
10    -2.151060 -2.051331  1.343838  3.397781      -3.850609  -0.963873
11    -3.568789 -5.727377  3.502572  0.233961      -3.182314  -0.963873
12     2.013334  3.603586  1.870582  2.899758      -2.959185  -0

In [27]:
# Удаляем значения вне доверительного интервала
std = df_std.std()

for_deletion = df_input_std.index[
    (df_input_std.Temperature < (-3 * std.Temperature)) |
    (df_input_std.Temperature > (3 * std.Temperature)) |
    (df_input_std.Humidity < (-3 * std.Humidity)) |
    (df_input_std.Humidity > (3 * std.Humidity)) |
    (df_input_std.Light < (-3 * std.Light)) |
    (df_input_std.Light > (3 * std.Light)) |
    (df_input_std.CO2 < (-3 * std.CO2)) |
    (df_input_std.CO2 > (3 * std.CO2)) |
    (df_input_std.HumidityRatio < (-3 * std.HumidityRatio)) |
    (df_input_std.HumidityRatio > (3 * std.HumidityRatio))
    ].tolist()

df_input_clean = df_input.drop(for_deletion)
df_input_clean_std = df_input_std.drop(for_deletion)

print(df_input_clean, '\n')
print(df_input_clean_std)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy
1     22.757693  25.605982  827.189941   475.785309       0.002475        0.0
8     24.064973  26.107119  372.435697  1400.546016       0.002795        0.0
13    21.702103  27.483594  816.453662  1104.540565       0.003832        0.0 

    Temperature  Humidity     Light       CO2  HumidityRatio  Occupancy
1      1.058139 -0.046189  2.268793 -0.977754      -2.613606  -0.963873
8      2.268092  0.153784  0.485553  2.054055      -2.108631  -0.963873
13     0.081138  0.703052  2.226692  1.083607      -0.473705  -0.963873


In [28]:
# Подаем полученные данные на вход классификатору

input_x = df_input_clean_std.iloc[:, :-1].values

input_y = classifier.predict(input_x)
probs = classifier.predict_proba(input_x)
probs = np.transpose(probs)

df_input_clean.Occupancy = input_y
df_input_clean['Probability0'] = probs[0]
df_input_clean['Probability1'] = probs[1]

print(df_input_clean)


    Temperature   Humidity       Light          CO2  HumidityRatio  Occupancy  \
1     22.757693  25.605982  827.189941   475.785309       0.002475          1   
8     24.064973  26.107119  372.435697  1400.546016       0.002795          0   
13    21.702103  27.483594  816.453662  1104.540565       0.003832          1   

    Probability0  Probability1  
1       0.688786      0.311214  
8       0.889049      0.110951  
13      0.210400      0.789600  
